In [6]:
# Packages

import pandas as pd
import os
import mne
import pytz
import plotly.graph_objects as go
from datetime import datetime, timedelta

RuntimeError: CPU dispatcher tracer already initlized

In [ ]:
# Helper functions
def unpack_edf(edf_path):
        
    base = os.path.basename(edf_path)  
    name, _ = os.path.splitext(base)
    out_csv = "./" + f"{name}_csv.csv"

    raw = mne.io.read_raw_edf(edf_path, preload=True, verbose='ERROR')

    ch_names = raw.ch_names
    data     = raw.get_data()              
    times    = raw.times                
    
    df = pd.DataFrame(data.T, columns=ch_names)
    df.insert(0, 'time_s', times)
        
    df.to_csv(out_csv, index=False)
    print(f"Saved {df.shape[0]} samples × {df.shape[1]} channels to '{out_csv}'")
    return df

def remove_col(file, col_name):
    df = pd.read_csv(file)
    df = df.drop(col_name, axis=1)
    df.to_csv(file, index=False)



Saved 1933300 samples × 13 channels to './EDF CSR003 13.06.2025_csv.csv'


,time_s,Resp nasal,Resp cpap,Resp chest,Resp abdomen,Position,Light,Pulse,SaO2 SpO2,Pulsewave,Sound microphone,Obstr,BR flow
0,0.00,1.541526e-14,-4.400000,100.000000,100.000000,153.000000,9990.000000,1.541526e-14,-5.395341e-14,-2.697671e-14,45.246052,-1.926908e-15,2.312289e-14
1,0.01,5.325330e+00,-4.299280,95.417350,95.450053,152.992405,9988.337169,4.405123e-02,1.678427e-02,2.059395e+01,28.610666,3.176170e-02,-1.850294e+00
2,0.02,1.276932e+01,-4.057712,88.502670,88.562663,152.951257,9983.386373,1.135815e-01,6.414891e-02,5.586023e+01,21.647974,6.014809e-02,-3.179044e+00
3,0.03,2.205695e+01,-3.688625,79.534150,79.614568,152.876576,9975.150552,2.085642e-01,1.420653e-01,5.931480e+01,50.080110,8.242404e-02,-3.985859e+00
4,0.04,3.280459e+01,-3.212671,68.910070,69.002764,152.768403,9963.634588,3.289579e-01,2.504868e-01,3.828183e+01,66.665141,9.647325e-02,-4.299275e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933295,19332.95,5.804051e+00,-2.904465,88.007535,87.791754,152.535334,9949.028117,1.618016e-01,3.745182e-01,5.918939e+01,49.864958,-1.056035e-01,1.507757e+01
1933296,19332.96,8.905736e-01,-3.419565,94.366407,94.201134,152.695123,9963.781053,7.858777e-02,2.386051e-01,7.070420e+01,49.851225,-9.997579e-02,1.159630e+01
1933297,19332.97,-2.434560e+00,-3.848965,99.078634,98.959285,152.821553,9975.260527,2.075937e-02,1.331439e-01,8.834156e+01,49.944305,-8.465816e-02,8.235215e+00
1933298,19332.98,-3.829842e+00,-4.167159,101.753410,101.676739,152.914545,9983.459750,-1.163474e-02,5.819648e-02,7.421988e+01,50.151827,-6.123005e-02,5.114930e+00


In [ ]:

# Functions to fix time stamp issues
def add_timestamp_export(csv_path: str, tz_name: str = 'Asia/Bishkek') -> None:
    """
    Reads a CSV file with a 'Time' column in 'DD.MM.YYYY HH:MM:SS' format,
    localizes each datetime to the specified timezone, computes a 'UnixTimestamp'
    column (seconds since epoch), inserts it immediately after the 'Duration' column,
    and overwrites the original file.
    
    Parameters
    ----------
    csv_path : str
        Path to the CSV file to read and overwrite.
    tz_name : str, optional
        Timezone name (default: 'Asia/Bishkek').
    """
    df = pd.read_csv(csv_path)
    tz = pytz.timezone(tz_name)
    dt_series = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M:%S', dayfirst=True)
    dt_series = dt_series.dt.tz_localize(tz)
    ts_series = (dt_series.astype('int64') // 10**6).astype(int)
    duration_idx = df.columns.get_loc('Duration') 
    df.insert(duration_idx + 1, 'UnixTimestamp', ts_series)

    df.to_csv(csv_path, index=False)

def add_timestamp_comments(csv_path: str, date_str: str, tz_name: str = 'Asia/Bishkek') -> None:
    """
    Reads a CSV file with a 'Time' column in 'HH:MM:SS' format (and other columns),
    combines each time with the provided date (YYYY-MM-DD), localizes to the specified
    timezone, computes a 'UnixTimestamp' column (seconds since epoch), inserts it 
    immediately after the 'Time' column, and overwrites the original file.
    
    Parameters
    ----------
    csv_path : str
        Path to the CSV file to read and overwrite.
    date_str : str
        Date string in 'YYYY-MM-DD' format to combine with each time.
    tz_name : str, optional
        Timezone name (default: 'Asia/Bishkek').
    """
    # Load CSV
    df = pd.read_csv(csv_path)
    tz = pytz.timezone(tz_name)
    dt_series = pd.to_datetime(date_str + ' ' + df['Time'], format='%Y-%m-%d %H:%M:%S')
    dt_series = dt_series.dt.tz_localize(tz)
    ts_series = (dt_series.astype('int64') // 10**9).astype(int)
    time_idx = df.columns.get_loc('Time')
    df.insert(time_idx + 1, 'UnixTimestamp', ts_series)
    df.to_csv(csv_path, index=False)

def add_timestamp_nasal(
    csv_path: str,
    start_datetime_str: str,
    tz_name: str = "Asia/Bishkek"
) -> None:
    """
    Reads a CSV whose first column is elapsed time in seconds (float or int) named arbitrarily,
    adds that many seconds onto the given start_datetime_str (YYYY-MM-DD HH:MM:SS),
    localizes to the specified timezone, computes Unix timestamps, and inserts them right
    after the elapsed‐seconds column. Finally, overwrites the CSV in place.

    Parameters
    ----------
    csv_path : str
        Path to the CSV file to read and overwrite.
    start_datetime_str : str
        The starting point (date + time) in 'YYYY-MM-DD HH:MM:SS' format.
        This should already be in the Bishkek zone (UTC+6).
    tz_name : str, optional
        Timezone name (default: 'Asia/Bishkek').
    """
    df = pd.read_csv(csv_path)

    elapsed_col = df.columns[0]

    tz = pytz.timezone(tz_name)

    base_dt_naive = pd.to_datetime(start_datetime_str, format="%Y-%m-%d %H:%M:%S")
    base_dt = tz.localize(base_dt_naive)
    elapsed_td = pd.to_timedelta(df[elapsed_col], unit="ms")

    dt_series = base_dt + elapsed_td
    unix_series = (dt_series.view("int64") // 10**9).astype(int)
    insert_idx = 1 
    df.insert(insert_idx, "UnixTimestamp", unix_series)
    df.to_csv(csv_path, index=False)


def bishkek_to_unix(date_str, time_str, elapsed_seconds):
    """
    Convert Bishkek time to Unix timestamp in milliseconds.
    
    Args:
        date_str (str): Date in DD-MM-YYYY format
        time_str (str): Time in HH:MM:SS format
        elapsed_seconds (int/float): Elapsed seconds from the start time
    
    Returns:
        int: Unix timestamp in milliseconds (milliseconds since epoch)
    """
    # Parse the date and time
    datetime_str = f"{date_str} {time_str}"
    dt = datetime.strptime(datetime_str, "%d-%m-%Y %H:%M:%S")
    
    # Set timezone to Bishkek (UTC+6)
    bishkek_tz = pytz.timezone('Asia/Bishkek')
    dt_bishkek = bishkek_tz.localize(dt)
    
    # Add elapsed seconds
    final_dt = dt_bishkek + timedelta(seconds=elapsed_seconds)
    
    # Convert to Unix timestamp in milliseconds
    unix_timestamp = int(final_dt.timestamp() * 1000)
    
    return unix_timestamp

def add_unix_timestamps_to_csv(csv_file_path, date_str, time_str):
    """
    Read CSV file, add Unix timestamps as second column based on elapsed seconds in first column,
    and update the same file.
    
    Args:
        csv_file_path (str): Path to CSV file to modify
        date_str (str): Start date in DD-MM-YYYY format
        time_str (str): Start time in HH:MM:SS format
    
    Returns:
        pandas.DataFrame: Modified dataframe with Unix timestamps
    """
    # Read the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Get the first column (elapsed seconds)
    elapsed_seconds_col = df.iloc[:, 0]
    
    # Calculate Unix timestamps for each row
    unix_timestamps = [bishkek_to_unix(date_str, time_str, elapsed) for elapsed in elapsed_seconds_col]
    
    # Insert Unix timestamp (in milliseconds) as the second column
    df.insert(1, 'UnixTimestamp', unix_timestamps)
    
    # Save back to the same file
    df.to_csv(csv_file_path, index=False)
    
    print(f"Modified CSV file: {csv_file_path}")
    print(f"Added {len(unix_timestamps)} Unix timestamps")
    
    return df

remove_col('../../data/cheyne-stokes/CSR003 13.06.2025/tabula-List of events CSR003 13.06.2025 (1).csv', 'UnixTimestamp')
add_timestamp_export('../../data/cheyne-stokes/CSR003 13.06.2025/tabula-List of events CSR003 13.06.2025 (1).csv')

In [ ]:
df1 = pd.read_csv("../../data/bishkek_csr/02_prepped/respeck/26-04-2025_respeck.csv")
df1['timestamp'] = pd.to_datetime(df1['interpolatedPhoneTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
df1['timestamp'] = df1['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
print(df1['timestamp'])
df1.set_index('timestamp', inplace=True)


df2 = pd.read_csv('../../data/cheyne-stokes/nasal_files/26-04-2025_nasal.csv')
df2['timestamp'] = pd.to_datetime(df2['UnixTimestamp'], unit='ms')
df2['timestamp'] = df2['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
print(df2['timestamp'])
df2.set_index('timestamp', inplace=True)

# Find the overlapping time range
overlap_start = max(df1.index.min(), df2.index.min())
overlap_end = min(df1.index.max(), df2.index.max())

print(f"Overlap period: {overlap_start} to {overlap_end}")
print(f"Overlap duration: {overlap_end - overlap_start}")

df1_overlap = df1[(df1.index >= overlap_start) & (df1.index <= overlap_end)]
df2_overlap = df2[(df2.index >= overlap_start) & (df2.index <= overlap_end)]

# Filter both dataframes to the overlap period
first_hour_end = overlap_start + pd.Timedelta(hours=2)

df1_first_hour = df1[(df1.index >= overlap_start) & (df1.index < first_hour_end)]
df2_first_hour = df2[(df2.index >= overlap_start) & (df2.index < first_hour_end)]

print(f"df1 overlap data points: {len(df1_overlap)}")
print(f"df2 overlap data points: {len(df2_overlap)}")

# Create the plot with overlapping data only
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1_first_hour.index, 
    y=df1_first_hour['breathingSignal'], 
    name='Breathing Signal (Respeck)',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=df2_first_hour.index, 
    y=df2_first_hour['Resp chest'],
    name='BR flow (Nasal)',
    mode='lines'
))

fig.update_layout(
    title=f'Comparison of Breathing Signals - Overlap Period<br>{overlap_start.strftime("%Y-%m-%d %H:%M:%S")} to {overlap_end.strftime("%Y-%m-%d %H:%M:%S")}',
    xaxis_title='Timestamp',
    yaxis_title='Signal Value',
    legend_title='Sensor',
    width=1200,
    height=600
)

fig.show()

0        2025-04-25 23:59:59.307000+06:00
1        2025-04-25 23:59:59.387000+06:00
2        2025-04-25 23:59:59.467000+06:00
3        2025-04-25 23:59:59.547000+06:00
4        2025-04-25 23:59:59.627000+06:00
                       ...               
432379   2025-04-26 10:12:39.492000+06:00
432380   2025-04-26 10:12:39.572000+06:00
432381   2025-04-26 10:12:39.652000+06:00
432382   2025-04-26 10:12:39.732000+06:00
432383   2025-04-26 10:12:39.812000+06:00
Name: timestamp, Length: 432384, dtype: datetime64[ns, Asia/Bishkek]


FileNotFoundError: [Errno 2] No such file or directory: '../../data/cheyne-stokes/nasal_files/26-04-2025_nasal.csv'

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import pytz
from datetime import datetime, timedelta
import numpy as np


RESPECK_FILE = '../../data/cheyne-stokes/CSR003 13.06.2025/Respeck_CSR003_35c5fa1801da28b5_D11B86C3EA87(6AL)_2025-06-13_decrypted.csv'
PSG_FILE = '../../data/cheyne-stokes/CSR003 13.06.2025/EDF CSR003 13.06.2025_csv.csv'
LABELS_FILE = '../../data/cheyne-stokes/CSR003 13.06.2025/tabula-List of events CSR003 13.06.2025 (1).csv'


def plot_apnea_events(respeck_df, psg_df, osa_events_df, buffer_minutes=2):
    """
    Plot PSG nasal respiration and Respeck sensor data during Obstructive Apnea events
    
    Parameters:
    - respeck_df: DataFrame with Respeck sensor data (x, y, z, breathingSignal)
    - psg_df: DataFrame with PSG data (including 'Resp nasal')
    - osa_events_df: DataFrame with Obstructive Apnea events
    - buffer_minutes: Minutes to show before and after each event
    """
    
    # Check if we have any OSA events
    if osa_events_df.empty:
        print("No Obstructive Apnea events found in the data")
        return
    
    # Calculate number of events and create subplots
    n_events = len(osa_events_df)
    print(f"Found {n_events} Obstructive Apnea events")
    
    # Create subplots
    fig = sp.make_subplots(
        rows=n_events, cols=1,
        subplot_titles=[f'Obstructive Apnea Event {i+1} - {event_time.strftime("%Y-%m-%d %H:%M:%S")}' 
                       for i, (event_time, _) in enumerate(osa_events_df.iterrows())],
        specs=[[{"secondary_y": True}] for _ in range(n_events)],
        vertical_spacing=0.08
    )
    
    buffer_td = timedelta(minutes=buffer_minutes)
    
    for idx, (event_time, event_row) in enumerate(osa_events_df.iterrows()):
        row = idx + 1
        
        # Define time window around the event
        start_time = event_time - buffer_td
        end_time = event_time + buffer_td
        
        # Extract event duration if available
        duration = event_row.get('Duration', 0)
        duration = float(duration.replace(',', '.'))

        if duration > 0:
            event_end_time = event_time + timedelta(seconds=duration)
        else:
            event_end_time = event_time + timedelta(seconds=30)  # Default 30s if no duration
        
        print(f"Processing event {idx+1} at {event_time}")
        print(f"Window: {start_time} to {end_time}")
        
        # Filter data for this time window
        respeck_window = respeck_df[start_time:end_time]
        psg_window = psg_df[start_time:end_time]
        
        if respeck_window.empty and psg_window.empty:
            print(f"No data found for event {idx+1} in the specified time window")
            # Add annotation for no data
            fig.add_annotation(
                text=f'No data available for Event {idx+1}<br>{event_time}',
                xref="x domain", yref="y domain",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=12),
                row=row, col=1
            )
            continue
        
        # Plot PSG nasal respiration on primary y-axis
        if not psg_window.empty and 'Resp nasal' in psg_window.columns:
            fig.add_trace(
                go.Scatter(
                    x=psg_window.index,
                    y=psg_window['Resp nasal'],
                    mode='lines',
                    name='PSG Nasal Resp',
                    line=dict(color='blue', width=2),
                    opacity=0.8,
                    showlegend=(idx == 0)
                ),
                row=row, col=1, secondary_y=False
            )
        
        # Plot Respeck data on secondary y-axis
        if not respeck_window.empty:
            # Plot x, y, z accelerometer data
            if 'x' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['x'],
                        mode='lines',
                        name='Respeck X',
                        line=dict(color='red', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            if 'y' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['y'],
                        mode='lines',
                        name='Respeck Y',
                        line=dict(color='green', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            if 'z' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['z'],
                        mode='lines',
                        name='Respeck Z',
                        line=dict(color='magenta', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            
            # Plot breathing signal with different style
            if 'breathingSignal' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['breathingSignal'],
                        mode='lines',
                        name='Respeck Breathing',
                        line=dict(color='orange', width=2),
                        opacity=0.9,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
        
        # Add shaded region for apnea event
        fig.add_vrect(
            x0=event_time, x1=event_end_time,
            fillcolor="red", opacity=0.3,
            layer="below", line_width=0,
            annotation_text=f"Apnea Event ({duration:.1f}s)",
            annotation_position="top left",
            row=row, col=1
        )
        
        # Add vertical line at event start
        fig.add_vline(
            x=event_time, line_dash="dash", line_color="red",
            line_width=2, opacity=0.8,
            row=row, col=1
        )
        
        # Set x-axis range
        fig.update_xaxes(range=[start_time, end_time], row=row, col=1)
        
        # Update y-axis labels
        fig.update_yaxes(title_text="PSG Nasal Respiration", title_font_color="blue", 
                        row=row, col=1, secondary_y=False)
        fig.update_yaxes(title_text="Respeck Sensor Data", title_font_color="red", 
                        row=row, col=1, secondary_y=True)
    
    # Update layout
    fig.update_layout(
        height=600*n_events,
        title_text="Obstructive Apnea Events Analysis",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    # Update x-axis labels for bottom subplot only
    fig.update_xaxes(title_text="Time", row=n_events, col=1)
    
    fig.show()
    
    # Print summary statistics
    print(f"\n=== Summary ===")
    print(f"Total Obstructive Apnea events: {n_events}")
    print(f"Time window per event: ±{buffer_minutes} minutes")
    
    if not osa_events_df.empty:
        durations = osa_events_df['Duration'].dropna()
        durations = pd.to_numeric(osa_events_df['Duration'].astype(str).str.replace(',', '.', regex=False), errors='coerce').dropna()
        if not durations.empty:
            print(f"Average event duration: {durations.mean():.1f} seconds")
            print(f"Duration range: {durations.min():.1f} - {durations.max():.1f} seconds")

In [ ]:
respeck_df = pd.read_csv(RESPECK_FILE)
respeck_df['timestamp'] = pd.to_datetime(respeck_df['interpolatedPhoneTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
respeck_df['timestamp'] = respeck_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
print(respeck_df)
respeck_df.set_index('timestamp', inplace=True)

psg_df = pd.read_csv(PSG_FILE)
psg_df['timestamp'] = pd.to_datetime(psg_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
psg_df['timestamp'] = psg_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
# print(psg_df['timestamp'])
psg_df.set_index('timestamp', inplace=True)

labels_df = pd.read_csv(LABELS_FILE)
labels_df['timestamp'] = pd.to_datetime(labels_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
labels_df['timestamp'] = labels_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
# print(labels_df['timestamp'])
labels_df.set_index('timestamp', inplace=True)

NameError: name 'pd' is not defined

In [ ]:


plot_apnea_events(respeck_df, psg_df, labels_df[labels_df['Event'] == 'Cheyne-Stokes'], buffer_minutes=4)

NameError: name 'plot_apnea_events' is not defined

In [ ]:
def plot_overview_with_events(respeck_df, psg_df, osa_events_df, max_hours=2):
    """
    Create an overview plot showing the full timeline with event markers
    """
    # Create subplots with secondary y-axis
    fig = sp.make_subplots(
        rows=2, cols=1,
        subplot_titles=('PSG Nasal Respiration with Apnea Events', 
                       'Respeck Sensor Data with Apnea Events'),
        vertical_spacing=0.1
    )
    
    # Limit data to prevent overcrowding
    if len(psg_df) > 0:
        start_time = psg_df.index.min()
        end_time = min(start_time + timedelta(hours=max_hours), psg_df.index.max())
        psg_subset = psg_df[start_time:end_time]
        respeck_subset = respeck_df[start_time:end_time]
    else:
        psg_subset = psg_df
        respeck_subset = respeck_df
    
    # Plot PSG data
    if not psg_subset.empty and 'Resp nasal' in psg_subset.columns:
        fig.add_trace(
            go.Scatter(
                x=psg_subset.index,
                y=psg_subset['Resp nasal'],
                mode='lines',
                name='PSG Nasal Resp',
                line=dict(color='blue', width=1),
                opacity=0.7
            ),
            row=1, col=1
        )
    
    # Plot Respeck data
    if not respeck_subset.empty:
        if 'breathingSignal' in respeck_subset.columns:
            fig.add_trace(
                go.Scatter(
                    x=respeck_subset.index,
                    y=respeck_subset['breathingSignal'],
                    mode='lines',
                    name='Breathing Signal',
                    line=dict(color='orange', width=1),
                    opacity=0.8
                ),
                row=2, col=1
            )
        if 'x' in respeck_subset.columns:
            fig.add_trace(
                go.Scatter(
                    x=respeck_subset.index,
                    y=respeck_subset['x'],
                    mode='lines',
                    name='X-axis',
                    line=dict(color='red', width=0.5),
                    opacity=0.6
                ),
                row=2, col=1
            )
        if 'y' in respeck_subset.columns:
            fig.add_trace(
                go.Scatter(
                    x=respeck_subset.index,
                    y=respeck_subset['y'],
                    mode='lines',
                    name='Y-axis',
                    line=dict(color='green', width=0.5),
                    opacity=0.6
                ),
                row=2, col=1
            )
        if 'z' in respeck_subset.columns:
            fig.add_trace(
                go.Scatter(
                    x=respeck_subset.index,
                    y=respeck_subset['z'],
                    mode='lines',
                    name='Z-axis',
                    line=dict(color='magenta', width=0.5),
                    opacity=0.6
                ),
                row=2, col=1
            )
    
    # Mark apnea events on both plots
    for event_time, event_row in osa_events_df.iterrows():
        if start_time <= event_time <= end_time:
            fig.add_vline(
                x=event_time, line_dash="dash", line_color="red",
                line_width=2, opacity=0.8,
                row=1, col=1
            )
            fig.add_vline(
                x=event_time, line_dash="dash", line_color="red",
                line_width=2, opacity=0.8,
                row=2, col=1
            )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Sleep Study Overview with Apnea Events",
        showlegend=True
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="PSG Nasal Respiration", row=1, col=1)
    fig.update_yaxes(title_text="Respeck Sensor Data", row=2, col=1)
    fig.update_xaxes(title_text="Time", row=2, col=1)
    
    fig.show()

# Usage example:
# Assuming your data is already loaded as shown in your code
plot_overview_with_events(respeck_df, psg_df, labels_df[labels_df['Event'] == 'Cheyne-Stokes'])

NameError: name 'respeck_df' is not defined